In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
#read data from csv files
how_much = pd.read_csv('items_number.csv', sep = ';')
items = pd.read_csv('items.csv', sep = ';')
paths = pd.read_csv('paths.csv', sep = ';')
machines = pd.read_csv('machines.csv', sep = ',')
norm_times = pd.read_csv('norm_time.csv', sep = ';')
setup_times = pd.read_csv('setup_time.csv', sep = ';')

In [3]:
#some setups to correct data
setup_times.fillna(value = 10, inplace = True) #fill blank fields
norm_times.fillna(value = '-', inplace = True)
paths.fillna(value = -1, inplace = True)
how_much = how_much[0:72] #cut useless definitions

pd.options.display.max_rows = 100
N_DETAILS = 72
N_MACHINES = 10
N_COMPLEX_DETAILS = 26

In [4]:
machines.head(15)
unique_machines = {'Пила' : 0, 'Плазменная' : 1, 'Токарный' : 2, 'Фрезерный' : 3, 
                   'Сверление' : 4, 'Сварка' : 5, 'Окраска' : 6}
paths.replace(unique_machines, inplace = True)
paths.head()

,№,Номенклатура,Тип оборудования,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,-1.0,-1,операция 1,операция 2,операция 3,операция 4,операция 5
1,1.0,Переходник 1,5,6,-1,-1,-1
2,1.1,Муфта А1,0,2,3,4,-1
3,1.2,Муфта А2,0,2,3,4,-1
4,2.0,Переходник 2,5,6,-1,-1,-1


In [5]:
class step:
    def __init__(self, machine, time, setup):
        self.machine_ = machine
        self.time_ = time
        self.setup_ = setup

In [6]:
class detail_type:
    def __init__(self, n, path, n_steps): #path - is an array of class step
        self.number_ = n
        self.n_steps_ = n_steps
        self.path_ = path

In [7]:
class dtype_state:
    def __init__(self, detail_type, n_to_do):
        self.detail_type_ = detail_type
        self.n_to_do_ = n_to_do
        self.done_ = 0
        self.cur_step_ = 0
        self.is_being_processed_ = 0
        self.details_progress_ = [detail(detail_type.n_steps_) for i in range(n_to_do)] #array of details
        self.needed_details_ = [] #-1 if simple detail; array of 'sub'-details if complex
        self.remaining_time_ = 0

In [8]:
class detail:
    def __init__(self, n_steps):
        self.progress_ = [0 for i in range(n_steps)]

In [9]:
State = [] #array of dtype_states

for i in range(N_DETAILS):        # in this cycle we initialize State
    #make path for the type
    j = 2
    path = []
    machine_type = paths.iloc[i + 1, j]
    while machine_type != -1:
        one_step = step(machine_type, 
                    norm_times.iloc[i, machine_type + 2],
                   setup_times.iloc[i, machine_type + 2])
        path.append(one_step)
        j += 1
        if j > 6:
            break
        machine_type = paths.iloc[i + 1, j]
        
    this_type = detail_type(i, path, j - 2)
    
    n_to_do = how_much['Кол-во'][i]
    state_elem = dtype_state(this_type, int(n_to_do))
    State.append(state_elem)
    
for i in range(N_DETAILS):
    rest = int ((how_much.iloc[i, 0] * 10) % 10)
    if rest != 0:
        State[i - rest].needed_details_.append(i)

for i in range(N_DETAILS):
    time_per_detail = 0
    for j in range(State[i].detail_type_.n_steps_):
        time_per_detail += State[i].detail_type_.path_[j].time_
    State[i].remaining_time = time_per_detail * State[i].n_to_do_
    
for i in range(N_DETAILS):
    print (i, ":", State[i].needed_details_)
    
# for i in range(N_DETAILS):
#     print ("------", i)
#     for j in range (len (State[i].detail_type_.path_)):
#         print (int (State[i].detail_type_.path_[j].machine_))


0 : [1, 2]
1 : []
2 : []
3 : [4, 5]
4 : []
5 : []
6 : [7, 8, 9, 10, 11, 12]
7 : []
8 : []
9 : []
10 : []
11 : []
12 : []
13 : [14, 15, 16, 17, 18, 19]
14 : []
15 : []
16 : []
17 : []
18 : []
19 : []
20 : [21, 22, 23, 24, 25, 26]
21 : []
22 : []
23 : []
24 : []
25 : []
26 : []
27 : []
28 : []
29 : []
30 : []
31 : []
32 : []
33 : []
34 : []
35 : []
36 : []
37 : []
38 : []
39 : []
40 : []
41 : []
42 : []
43 : []
44 : []
45 : []
46 : []
47 : []
48 : []
49 : []
50 : []
51 : []
52 : []
53 : []
54 : []
55 : []
56 : []
57 : []
58 : []
59 : []
60 : []
61 : []
62 : []
63 : []
64 : []
65 : []
66 : []
67 : []
68 : []
69 : []
70 : []
71 : []


In [10]:
def full_time ():
    fulltime = 0
    for i in range (len (State)):
        fulltime += State[i].remaining_time
    return fulltime

print (full_time())

25277.0


In [11]:
class machine:
    def __init__(self, mtype):
        self.machine_type_ = mtype
        self.processing_detail_ = [-1, -1] #[0] - type of the detail and [1] - number in details_progress array
        self.busy = 0
        self.time_setuped = 0
        
    # machine does not start before previous work haven't finished
    def what_available(self):
        available_details = []
        for i in range(len(State)):
            if (State[i].is_being_processed_ == 0) and (State[i].done_ == 0):
                ready = 1  
                for j in range(len(State[i].needed_details_)):
                    if (State[State[i].needed_details_[j]].done_ == 0):
                        ready = 0
                        break
#                 if (State[i].cur_step_ > 0):
#                     print ("i = ", i, "cur_step = ", State[i].cur_step_, "len = ", len(State[i].detail_type_.path_))
                if (State[i].detail_type_.path_[State[i].cur_step_].machine_ == self.machine_type_) and (ready == 1):
                    available_details.append(i)

        
        return available_details
    
    def long_job_first(self):
        available = self.what_available()
        if (len(available) == 0): #there is nothing to do
            return -1
        
        best = available[0]
        for i in range(len(available)):
            if (State[available[i]].remaining_time +
                State[available[i]].detail_type_.path_[State[available[i]].cur_step_].setup_ >
                State[best].remaining_time +
                State[best].detail_type_.path_[State[best].cur_step_].setup_):
                best = available[i]
        
        return best
    
    def short_job_first(self):
        available = self.what_available()
        if (len(available) == 0): #there is nothing to do
            return -1
        
        best = available[0]
        for i in range(len(available)):
            if (State[available[i]].remaining_time +
                State[available[i]].detail_type_.path_[State[available[i]].cur_step_].setup_ <
                State[best].remaining_time +
                State[best].detail_type_.path_[State[best].cur_step_].setup_):
                best = available[i]
        
        return best
    
    def first_come_first_served(self):
        available = self.what_available()
        if (len(available) == 0): #there is nothing to do
            return -1
        
        best = available[0]
        
        return best
    
    def first_come_last_served(self):
        available = self.what_available()
        if (len(available) == 0): #there is nothing to do
            return -1
        
        best = available[-1]
        
        return best
    
    def first_come_random_served(self):
        available = self.what_available()
        if (len(available) == 0): #there is nothing to do
            return -1
        
        best = available[np.random.randint(0, len(self.what_available()))]
        
        return best
    
    def short_in_complex_first(self):
        available = self.what_available()
        if (len(available) == 0): #there is nothing to do
            return -1
        
        best = available[0]
        first_single = 0
        
        for i in range (len(available)):
            if State[available[i]].detail_type_.number_ <= N_COMPLEX_DETAILS:
                first_single = i;
                break
                
        #print ("first_single = ", first_single)
            
        if (first_single != 0):
            for i in range(first_single):
                if (State[available[i]].remaining_time +
                    State[available[i]].detail_type_.path_[State[available[i]].cur_step_].setup_ <
                    State[best].remaining_time +
                    State[best].detail_type_.path_[State[best].cur_step_].setup_):
                    best = available[i]
#             print ("best =", best)
            return best
        else:
            for i in range(len(available)):
                if (State[available[i]].remaining_time +
                    State[available[i]].detail_type_.path_[State[available[i]].cur_step_].setup_ <
                    State[best].remaining_time +
                    State[best].detail_type_.path_[State[best].cur_step_].setup_):
                    best = available[i]
#             print ("best =", best)
            return best
    
    
    def processing (self, next_detail_choice):
        if (self.busy == 0):
            #self.processing_detail_[0] = self.long_job_first()
            prev_detail = self.processing_detail_[0]
            self.processing_detail_[0] = next_detail_choice()
            self.processing_detail_[1] = 0
            if (self.processing_detail_[0] != -1):
            
                if (self.processing_detail_[0] != prev_detail):
                    self.busy = 1
                else:
                    self.busy = 2
                State[self.processing_detail_[0]].is_being_processed_ = 1
                
        if (self.busy == 1):
            if (self.time_setuped ==
                State[self.processing_detail_[0]].detail_type_.path_[State[self.processing_detail_[0]].cur_step_].setup_):
                self.busy = 2
                self.time_setuped = 0 
                
            else:
                self.time_setuped += 1
                return self.processing_detail_[0]
      
                
        if (self.busy == 2):
            i = self.processing_detail_[0]
            
            State[i].details_progress_[self.processing_detail_[1]].progress_[State[i].cur_step_] += 1
            State[i].remaining_time_ -= 1
#             if (self.machine_type_ != State[i].detail_type_.path_[State[i].cur_step_].machine_):
#                 print ('\033[91m', "mType = ", self.machine_type_, "wantedType = ", State[i].detail_type_.path_[State[i].cur_step_].machine_, '\033[0m')
#             else:
#                 print ("mType = ", self.machine_type_, "wantedType = ", State[i].detail_type_.path_[State[i].cur_step_].machine_)
            if (State[i].detail_type_.path_[State[i].cur_step_].time_ == 
                State[i].details_progress_[self.processing_detail_[1]].progress_[State[i].cur_step_]):
            # если процедура над деталью завершилась
                
                if (self.processing_detail_[1] != State[i].n_to_do_ - 1):
                # если это НЕ последняя деталь в партии
                    self.processing_detail_[1] += 1
                
                else:
                # если это последняя деталь в партии
                    self.busy = 0
                    State[self.processing_detail_[0]].is_being_processed_ = 0
                    if (State[i].cur_step_ == State[i].detail_type_.n_steps_ - 1):
                    # если это последний этап последней детали
                        State[i].done_ = 1
#                         State.pop(i)
                    else:
                    # если не последний этап последней детали
                        State[i].cur_step_ += 1
                
            return i
        
        return -1
#                 if (State[i].cur_step_ == State[i].detail_type_.n_steps_):
#                 # если это последний этап данной детали
#                     if (self.processing_detail_[1] == State[i].n_to_do - 1):
#                     # если это последний этап последней детали
#                         State[i].done_ = 1
#                         State[i].cur_step_ += 1
#                         self.busy = 0
                
                
        

In [12]:
machineA = machine (3)
print (machineA.what_available())

[7, 14, 61, 63, 66, 68, 70]


In [13]:
Factory = []

for i in range (N_MACHINES):
    mtype = unique_machines[machines.iloc[i, 2]]
    Factory.append(machine(mtype))
    
def finish ():
    for i in range (len (State)):
        if State[i].done_ == 0:
            return 1
    return 0

In [14]:
History = np.arange(0).reshape(N_MACHINES, -1)
#fill Histroy with -1
History[0:N_MACHINES] = -1
History

array([], shape=(10, 0), dtype=int64)

In [15]:
# shortest job first

t = 0
while (finish()):
    machine_state = np.repeat(-1, N_MACHINES).reshape(N_MACHINES, -1)
    for k in range (len (Factory)):
        machine_state[k] = Factory[k].processing(Factory[k].first_come_first_served)
        
    History = np.hstack((History, machine_state))
    t += 1

print ("T = ", t)
print ("history_size", History.shape)
print ("time ", History.shape[1])
History

T =  4495
history_size (10, 4495)
time  4495


array([[ 1,  1,  1, ..., -1, -1, -1],
       [24, 24, 24, ..., -1, -1, -1],
       [62, 62, 62, ..., -1, -1, -1],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]])

In [16]:
History_Frame = pd.DataFrame(History)
History_Frame.to_csv('history.csv')

In [17]:
def zero_line (line):
    for i in range (len(line)):
        if line[i] == 1:
            return 0
    return 1


    machine_num = 0
    
for machine_num in range (N_MACHINES):    
    Machine_schedule = np.zeros((N_DETAILS, History.shape[1]))
    New_machine_schedule = np.arange(0).reshape(-1, History.shape[1] + 1)

    for i in range (History.shape[1]):
        if (History[machine_num, i] >= 0):
            Machine_schedule[History[machine_num, i], i] = 1

    for i in range (Machine_schedule.shape[0]):
        if (zero_line(Machine_schedule[i, :]) == 0):
            detail = np.hstack((np.array([i]), Machine_schedule[i, :]))
            New_machine_schedule = np.vstack((New_machine_schedule, detail))

    Machine_schedule_frame = pd.DataFrame(New_machine_schedule)
    string = "machine__schedules/machine_schedule" + str (machine_num) + ".csv" 
    Machine_schedule_frame.to_csv(string)
    
    Final_machine_schedule = New_machine_schedule [:, 0:-1:5]
    
    Machine_schedule_frame = pd.DataFrame(Final_machine_schedule)
    string = "easier_machine__schedules/easy_machine_schedule" + str (machine_num) + ".csv" 
    Machine_schedule_frame.to_csv(string)
        
# New_machine_schedule
    

In [18]:
# def highlight_max(s):
#     return 'background-color: white' if s == 0 else 'background-color: green'
# History_Frame

In [19]:
# Machine_schedule_frame.style.applymap(highlight_max)

In [20]:
# # longest job first

# t = 0
# while (finish()):
#     #print ("len(State) = ", len(State))
#     for k in range (len (Factory)):
#         Factory[k].processing(Factory[k].short_job_first)
#     t += 1

# print ("T = ", t)
        